In [1]:
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

/bin/bash: python: command not found
/bin/bash: python: command not found


In [2]:
import random
import torch

random.seed(42)
torch.manual_seed(42)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [4]:
f = open('/content/drive/MyDrive/datasets/deu.txt')

lines = f.readlines()
random.shuffle(lines)

f.close()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/datasets/deu.txt'

In [ ]:
print(lines[0])

Tom is waiting in the hallway.	Tom wartet auf dem Flur.	CC-BY 2.0 (France) Attribution: tatoeba.org #4132396 (CK) & #4134744 (Pfirsichbaeumchen)



In [ ]:
len(lines)

271774

In [ ]:
PAIRS_LEN = 50000

pairs = {'en': [], 'de': []}

for line in lines[:PAIRS_LEN]:
  splits = line.split('\t')
  pairs['en'].append(splits[0])
  pairs['de'].append(splits[1])

In [ ]:
pairs['en'][:10]

['Tom is waiting in the hallway.',
 "It doesn't make much sense to me that I have to pay more income tax than you even though you make more money than I do.",
 'This palace was built for the rich king.',
 "Tom doesn't work here anymore.",
 "You're not at all what I expected.",
 'Tom is still a very good tennis player.',
 'We have to do this now.',
 'Tom was tickled pink.',
 'Tom and I have jobs.',
 'Everything went well.']

In [ ]:
pairs['de'][:10]

['Tom wartet auf dem Flur.',
 'Ich kann keinen Sinn darin erkennen, dass ich mehr Einkommenssteuer als du entrichten muss, wenngleich du mehr verdienst als ich.',
 'Dieser Palast wurde für den reichen König erbaut.',
 'Tom arbeitet nicht mehr hier.',
 'Du bist absolut nicht das, was ich erwartet habe.',
 'Tom ist immer noch ein sehr guter Tennisspieler.',
 'Wir müssen das jetzt tun.',
 'Tom war ganz aus dem Häuschen.',
 'Tom und ich, wir sind berufstätig.',
 'Alles lief gut ab.']

In [ ]:
TEST_PAIRS_LEN = 10000

test_pairs = {'en': [], 'de': []}

for line in lines[PAIRS_LEN:PAIRS_LEN+TEST_PAIRS_LEN]:
  splits = line.split('\t')
  test_pairs['en'].append(splits[0])
  test_pairs['de'].append(splits[1])

In [ ]:
test_pairs['en'][:10]

["I'm a translator.",
 'People used to write books using typewriters.',
 'We have a bit more work to do.',
 'Tom jumped across the mud puddle.',
 "The television doesn't work.",
 'I found out what Tom likes.',
 'Shut all the doors and windows.',
 "I don't like it when you're so busy.",
 "I haven't yet finished my homework.",
 "Tom doesn't seem to be responding to the treatment."]

In [ ]:
test_pairs['de'][:10]

['Ich bin Übersetzerin.',
 'Früher hat man Bücher mit der Schreibmaschine geschrieben.',
 'Wir haben noch ein bisschen mehr Arbeit zu erledigen.',
 'Tom sprang über die Schlammpfütze.',
 'Der Fernseher geht nicht.',
 'Ich weiß jetzt, was Tom mag.',
 'Mach alle Türen und Fenster zu.',
 'Es gefällt mir nicht, wenn du so beschäftigt bist.',
 'Ich bin noch nicht mit den Hausaufgaben fertig.',
 'Die Therapie scheint bei Tom nicht anzuschlagen.']

In [ ]:
from torchtext.data.utils import get_tokenizer

tokenizer_de = get_tokenizer('spacy', language='de_core_news_sm')
tokenizer_en = get_tokenizer('spacy', language='en_core_web_sm')

In [ ]:
tokenizer_en(pairs['en'][10123])

['Tom', 'loves', 'me', 'and', 'I', 'love', 'him', ',', 'too', '.']

In [ ]:
tokenizer_de(pairs['de'][10123])

['Tom', 'liebt', 'mich', ',', 'und', 'ich', 'liebe', 'ihn', 'auch', '.']

In [ ]:
def tokenizer_all(pairs):
  tokens = {'en': [], 'de': []}

  for pair in pairs['en']:
    tokens['en'].append(tokenizer_en(pair.lower()))

  for pair in pairs['de']:
    tokens['de'].append(tokenizer_en(pair.lower()))

  return tokens

In [ ]:
tokens = tokenizer_all(pairs)

In [ ]:
test_tokens = tokenizer_all(test_pairs)

In [ ]:
tokens['en'][10]

['he', 'did', 'it', 'at', 'his', 'own', 'risk', '.']

In [ ]:
pairs['en'][10]

'He did it at his own risk.'

In [ ]:
tokens['de'][10]

['er', 'hat', 'es', 'auf', 'eigene', 'gefahr', 'hin', 'getan', '.']

In [ ]:
pairs['de'][10]

'Er hat es auf eigene Gefahr hin getan.'

In [ ]:
from torchtext.vocab import build_vocab_from_iterator

vocab_en = build_vocab_from_iterator(tokens['en'], specials=['<sos>', '<eos>'], min_freq=2)
vocab_de = build_vocab_from_iterator(tokens['de'], specials=['<sos>', '<eos>'], min_freq=2)

In [ ]:
vocab_en['house']

145

In [ ]:
vocab_de['haus']

169

In [ ]:
vocab_en['<sos>'], vocab_en['<eos>']

(0, 1)

In [ ]:
vocab_de['<sos>'], vocab_de['<eos>']

(0, 1)

In [ ]:
len(vocab_en), len(vocab_de)

(5977, 8555)

In [ ]:
from torch.utils.data import Dataset

In [ ]:
class TranslationDataset(Dataset):
  def __init__(self, all_tokens, vocab_en, vocab_de):
    self.all_tokens = all_tokens
    self.data_en = []
    self.data_de = []

    for tokens in all_tokens['en']:
      tokens_index = [vocab_en[t] for t in tokens if t in vocab_en]
      self.data_en.append(tokens_index)

    for tokens in all_tokens['de']:
      tokens = ['<sos>'] + tokens + ['<eos>']
      tokens_index = [vocab_de[t] for t in tokens if t in vocab_de]
      self.data_de.append(tokens_index)

  def __len__(self):
    return len(self.all_tokens['en'])

  def __getitem__(self, index):
    return torch.LongTensor(self.data_en[index]), torch.LongTensor(self.data_de[index])

In [ ]:
train_dataset = TranslationDataset(tokens, vocab_en, vocab_de)
test_dataset = TranslationDataset(test_tokens, vocab_en, vocab_de)

In [ ]:
len(train_dataset)

50000

In [ ]:
train_dataset[10]

(tensor([  16,   28,   14,   42,   33,  335, 1210,    2]),
 tensor([   0,   15,   17,   13,   30, 1412,  851,  393,  170,    2,    1]))

In [ ]:
from torch.utils.data import DataLoader

dataloader_train = DataLoader(train_dataset)
dataloader_test = DataLoader(test_dataset)

In [ ]:
from torch import nn

class Encoder(nn.Module):
  def __init__(self, input_dim, embedding_dim, hidden_dim, n_layers):
    super().__init__()

    self.hidden_dim = hidden_dim
    self.n_layers = n_layers

    self.embedding = nn.Embedding(input_dim, embedding_dim)
    self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers)

  def forward(self, x):
    embedding_vector = self.embedding(x)
    outputs, (hidden, cell) = self.lstm(embedding_vector)

    return hidden, cell

In [ ]:
encoder = Encoder(len(vocab_en), 128, 64, 1)

In [ ]:
encoder(train_dataset[0][0])

(tensor([[ 0.1461, -0.3632,  0.0677,  0.0906, -0.1714,  0.0315, -0.1013,  0.1105,
          -0.2332, -0.1872, -0.0027, -0.1165, -0.2340,  0.2100, -0.0755, -0.0903,
          -0.1000, -0.2103,  0.0179, -0.1207,  0.0382,  0.0890, -0.3627,  0.0285,
           0.0129,  0.5908, -0.2163, -0.1351,  0.1478,  0.2832, -0.0817,  0.2473,
           0.1780,  0.1593,  0.0232, -0.2659,  0.0450,  0.3377,  0.0561,  0.0745,
          -0.1208, -0.0707, -0.2865, -0.1645,  0.3728, -0.1019, -0.0982, -0.6183,
          -0.2108, -0.2089,  0.0585, -0.0193, -0.3653,  0.1887, -0.0627,  0.0915,
           0.3717,  0.0217, -0.1198,  0.0597, -0.2428,  0.1848,  0.1379,  0.2247]],
        grad_fn=<SqueezeBackward1>),
 tensor([[ 0.3040, -1.0484,  0.1345,  0.3691, -0.2792,  0.0434, -0.2275,  0.4589,
          -0.3444, -0.3496, -0.0035, -0.2358, -0.3419,  0.3759, -0.2313, -0.2805,
          -0.2265, -0.3312,  0.0642, -0.2084,  0.0584,  0.1329, -1.0029,  0.0675,
           0.0338,  0.9334, -0.5109, -0.1589,  0.4538,  1.1

In [ ]:
class Decoder(nn.Module):
  def __init__(self, output_dim, embedding_dim, hidden_dim, n_layers):
    super().__init__()

    self.hidden_dim = hidden_dim
    self.n_layers = n_layers
    self.output_dim = output_dim

    self.embedding = nn.Embedding(output_dim, embedding_dim)
    self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers)
    self.linear = nn.Linear(hidden_dim, output_dim)

  def forward(self, x, hidden, cell):
    embedding_vector = self.embedding(x)
    outputs, (hidden, cell) = self.lstm(embedding_vector, (hidden, cell))
    prediction = self.linear(hidden)

    return prediction, hidden, cell

In [ ]:
decoder = Decoder(len(vocab_de), 128, 64, 1)

In [ ]:
decoder(torch.LongTensor([0]), torch.zeros(1,64), torch.zeros(1,64))

(tensor([[-0.1225, -0.1167,  0.0330,  ...,  0.0748, -0.0383, -0.0926]],
        grad_fn=<AddmmBackward0>),
 tensor([[-0.0553, -0.0289, -0.0626, -0.2369,  0.0931, -0.2649,  0.2655, -0.0157,
          -0.1895, -0.1397, -0.0195, -0.1597, -0.1251, -0.1065,  0.1602,  0.0326,
          -0.0566,  0.1228,  0.0470, -0.0673,  0.0788, -0.1788, -0.0017,  0.1333,
           0.1611,  0.1788, -0.1205, -0.0440,  0.0716, -0.2015, -0.0080,  0.0088,
           0.0600,  0.2735, -0.0434,  0.0469,  0.0662,  0.0424,  0.0464,  0.1469,
          -0.1839, -0.0130, -0.1696, -0.1214,  0.0995, -0.0760,  0.1314, -0.0568,
          -0.1814, -0.1904, -0.1264, -0.0413,  0.0451, -0.0133, -0.0959, -0.0169,
           0.0438, -0.0351,  0.3722, -0.0840, -0.3564, -0.1152,  0.1002, -0.0774]],
        grad_fn=<SqueezeBackward1>),
 tensor([[-0.1439, -0.1299, -0.1404, -0.5240,  0.3439, -0.4127,  0.4731, -0.0651,
          -0.3292, -0.4471, -0.0218, -0.2797, -0.2041, -0.1697,  0.6835,  0.2540,
          -0.1742,  0.3457,  0.128

In [ ]:
class Seq2Seq(nn.Module):
  def __init__(self, encoder, decoder, device):
    super().__init__()

    self.encoder = encoder
    self.decoder = decoder
    self.device = device

  def forward(self, source, target, teacher_forcing_ratio=0.5):
    source = source.to(self.device)
    target = target.to(self.device)

    target_len = target.shape[0]
    target_vocab_size = self.decoder.output_dim

    outputs = torch.zeros(target_len, 1, target_vocab_size).to(self.device)
    outputs[0][0][0] = 1  # O token inicial é o <sos> que tem índice 0.

    hidden, cell = self.encoder(source)

    input = target[0]
    input = input.unsqueeze(0)

    for t in range(1, target_len):
      prediction, hidden, cell = self.decoder(input, hidden, cell)

      outputs[t] = prediction
      top1 = prediction.argmax(1)

      teacher_force = random.random() < teacher_forcing_ratio

      input = target[t].unsqueeze(0) if teacher_force else top1

    return outputs

In [ ]:
x = torch.tensor([1,2,3])
x

tensor([1, 2, 3])

In [ ]:
x.shape

torch.Size([3])

In [ ]:
x.unsqueeze(0)

tensor([[1, 2, 3]])

In [ ]:
x.unsqueeze(0).shape

torch.Size([1, 3])

In [ ]:
device = torch.device('cuda')
seq2seq = Seq2Seq(encoder, decoder, device).to(device)

In [ ]:
from torch import optim

optimizer = optim.Adam(seq2seq.parameters())
loss_fn = nn.CrossEntropyLoss()

In [ ]:
from tqdm.notebook import tqdm

def train():
  seq2seq.train()

  epoch_loss = 0

  for source, target in tqdm(dataloader_train):
    optimizer.zero_grad()

    source = source.to(device)
    target = target.to(device)

    # print('source', source)
    # print('source shape', source.shape)

    source = source.squeeze(0)
    target = target.squeeze(0)
    print(source)

    # print('source squeeze', source)
    # print('source shape squeeze', source.shape)

    output = seq2seq(source, target)

    # print('output', output)
    # print('output shape', output.shape)

    output = output[1:]
    target = target[1:]

    output = output.permute(1, 0, 2)
    output = output.squeeze(0)

    # print('output', output)
    # print('output shape', output.shape)

    # print('target', target)
    # print('target shape', target.shape)

    loss = loss_fn(output, target)

    # print('loss', loss)
    # print('loss shape', loss.shape)

    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()

    # break

  return epoch_loss/len(dataloader_train)


In [ ]:
def test():
  seq2seq.eval()
  epoch_loss = 0

  with torch.no_grad():
    for source, target in tqdm(dataloader_test):

      source = source.to(device)
      target = target.to(device)

      source = source.squeeze(0)
      target = target.to(device)
      target = target.squeeze(0)

      output = seq2seq(source, target)

      output = output[1:]
      target = target[1:]

      output = output.permute(1, 0, 2)
      output = output.squeeze(0)

      loss = loss_fn(output, target)

      epoch_loss += loss.item()

  return epoch_loss/len(dataloader_test)

In [ ]:
N_EPOCHS = 4
train_losses = []
test_losses = []

for epoch in range(N_EPOCHS):
  print('EPOCH', epoch)

  train_loss = train()
  train_losses.append(train_loss)
  print('train loss:', train_loss)

  test_loss = test()
  test_losses.append(test_loss)
  print('test loss:', test_loss)



EPOCH 0


  0%|          | 0/50000 [00:00<?, ?it/s]

train loss: 4.293588434118629


  0%|          | 0/10000 [00:00<?, ?it/s]

test loss: 3.83805919188261
EPOCH 1


  0%|          | 0/50000 [00:00<?, ?it/s]